In [ ]:
! pip install tensorflow_recommenders

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfr

In [ ]:
ratings = tfds.load("movielens/100k-ratings",split="train")
movies = tfds.load("movielens/100k-movies",split="train")

In [4]:
## taking 2 features id and movie title from ratings

ratings = ratings.map(lambda x:{
    "movie_title":x["movie_title"],
    "user_id":x["user_id"]
})
movies = movies.map(lambda x:x["movie_title"])

In [5]:
# building vocabularies to convert data into integer indicies for embedding

user_ids_vocabulary= tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
movie_titles_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary.adapt(movies)

In [9]:
class TwoTowerMovielensModel(tfr.Model):
  ### prediction model
  def __init__(self, user_model: tf.keras.Model,movie_model: tf.keras.Model,task:tfr.tasks.Retrieval):
    super().__init__()
  
   # setting up movie and user model
    self.user_model = user_model
    self.movie_model = movie_model
     
    ## determining which task the model is going to perform..retrieval,ranking or post ranking
    self.task = task

  def compute_loss(self,features,training = False) -> tf.Tensor:
    user_embeddings = self.user_model(features['user_id'])
    movies_embeddings = self.movie_model(features["movie_title"])
    return self.task(user_embeddings,movies_embeddings)


In [11]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(),64)
])
movies_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(),64)
])
task = tfr.tasks.Retrieval(metrics=tfr.metrics.FactorizedTopK(movies.batch(128).map(movies_model)))

In [12]:
model = TwoTowerMovielensModel(user_model,movies_model,task)
model.compile(optimizer = tf.keras.optimizers.Adagrad(0.5))

In [14]:
model.fit(ratings.batch(4096),epochs=10)

Epoch 1/10
25/25 [==============================] - 31s 1s/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0108 - factorized_top_k/top_10_categorical_accuracy: 0.0287 - factorized_top_k/top_50_categorical_accuracy: 0.1773 - factorized_top_k/top_100_categorical_accuracy: 0.3138 - loss: 30075.3933 - regularization_loss: 0.0000e+00 - total_loss: 30075.3933
Epoch 2/10
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0117 - factorized_top_k/top_10_categorical_accuracy: 0.0319 - factorized_top_k/top_50_categorical_accuracy: 0.1996 - factorized_top_k/top_100_categorical_accuracy: 0.3449 - loss: 29820.5883 - regularization_loss: 0.0000e+00 - total_loss: 29820.5883
Epoch 3/10
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 9.1000e-04 - factorized_top_k/top_5_categorical_a

In [15]:
### predictions using brute force to search to set up retrival using the trained representation

In [16]:
index =tfr.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(movies.batch(100).map(lambda title:(title, model.movie_model(title))))

In [18]:
import numpy as np

In [19]:
_, titles = index(np.array(['42']))
print(f'top 3 recommendations for user 42:{titles[0,:3]}')

top 3 recommendations for user 42:[b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']
